In [1]:
import networkx as nx
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 

In [2]:
## Multiprocessing Package
from multiprocessing import cpu_count
from dask.distributed import Client, progress
import dask
client = Client(threads_per_worker=2)
client

Client Scheduler: tcp://127.0.0.1:33781 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 33.60 GB


In [3]:
#########################################################################################
############################### Network Helper Functions  ###############################
#########################################################################################

'''
    Global Variables
    N      ::  Number of nodes
    phi    ::  Threshold value
    label  ::  Attribute name of influence
    time   ::  Attribute name of time
'''

def set_time(G, value, node=None):
    '''
        Set the time of an individual node in a network G 
        to value or set the time of all nodes to value.
        G      ::  a networkx graph
        node   ::  a reference to a node in G
        value  ::  a non-negative integer
    '''
    if node:
        G.nodes[node][time] = value
    else:
        time_attrib = {i : value for i in range(N)}
        nx.set_node_attributes(G,time_attrib, time)


def set_influence(G, value, node=None):
    '''
        Set the influence of an individual node in a network G 
        to value or set the influence of all nodes to value.
        G      ::  a networkx graph
        node   ::  a reference to a node in G
        value  ::  an integer 0 or 1
    '''
    if node:
        G.nodes[node][label] = value
    else:
        influence_attrib = { i : value for i in range(N) }
        nx.set_node_attributes(G,influence_attrib, label)
        
def get_is_influenced(G, node):
    '''
        Returns if node in G is influenced.
    '''
    return G.nodes[node][label]
        
def get_number_influenced(G):
    '''
        Get the number of influenced nodes.
    '''
    return sum(nx.get_node_attributes(G, label).values())

#################################################################################
########################## Simulation Helper Functions ##########################
#################################################################################

def get_uninfluenced_neighbours(G, nodes):
    '''
        Return a set of neighbours of nodes
        that are uninfluenced.
    '''
    neighbours = set()
    for node in nodes:
        friends = list(G.neighbors(node))
        neighbours.update([friend for friend in friends if G.nodes[friend][label] == 0])
    return neighbours

def update_influence(G, node, phi, time):
    '''
        Update a node's influence status.
    '''
    if get_is_influenced(G, node) == 1:
        return True
    else:
        friends = list(G.neighbors(node))
        num_friends = len(friends)
        
        if num_friends == 0:
            return False
        
        num_influenced = sum([1 for friend in friends if G.nodes[friend][label] == 1])
        if num_influenced/num_friends > phi:
            set_influence(G, 1, node)
            set_time(G, time, node)
            return True
        return False
    
def simulate_spread(G, initial_node, phi):
    G_tmp = G.copy()
    set_influence(G_tmp, 1, initial_node)

    t = [0 for _ in range(N)]
    time, num_influenced = 1, 1
    t[0] = 1
    influenced_nodes = set([initial_node])
    
    while num_influenced > 0:
        num_influenced = 0
        neighbours = get_uninfluenced_neighbours(G_tmp, influenced_nodes)
        for node in neighbours:
            if update_influence(G_tmp, node, phi, time):
                num_influenced += 1
                influenced_nodes.add(node)
        t[time] = num_influenced
        time += 1
    return (len(influenced_nodes), t)

In [12]:
#################################################################################
############################# Simulation  Functions #############################
#################################################################################

def run_simulation_RG(p):
    '''
        Simulation of Poisson/Binomial Random Graph
    '''
    G = nx.erdos_renyi_graph(N,p)
    set_influence(G, 0)
    set_time(G, 0)
    
    ## Retrieve influential nodes - top q% and non-influential nodes
    degree_ordered_nodes = sorted(list(G.nodes()), key=lambda x: G.degree(x), reverse=True)
    influential_nodes = degree_ordered_nodes[:int(q*N)]
    normal_nodes = degree_ordered_nodes[int(q*N):]
        
    influential_S = []
    influential_t = []
    normal_S = []
    normal_t = []
    
    for node in influential_nodes:
        S, t = simulate_spread(G, node, phi)
        influential_S.append(S)
        influential_t.append(t)
        
    for node in normal_nodes:
        S, t = simulate_spread(G, node, phi)
        normal_S.append(S)
        normal_t.append(t)
    
    t_expected_normal = sum([i * normal_t[i] for i in range(len(normal_t))])/N
    t_expected_influential = sum([i * influential_t[i] for i in range(len(influential_t))])/N
    
    return [np.mean(influential_S), np.mean(normal_S), t_expected_normal, t_expected_influential]

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


# Simulation of Poisson Random Graph

The following code is for simulating and measuring the cascade of influentials.

In [9]:
######################################################################
############################# Parameters #############################
######################################################################

N = 1000
q = 0.1   
phi = 0.18
max_n_avg = 7
increment = 0.2
num_simulations = 100
label = 'is_influenced'
time = 'time'

n_avg = np.arange(1, max_n_avg, increment)
p = [avg/(N-1) for avg in n_avg]
n = len(p)

In [10]:
%%time
pool = []
for i in tqdm(range(num_simulations)):
    for j in range(n):
        pool.append(dask.delayed(run_simulation_RG)(p[j]))

results = dask.compute(pool)

100%|██████████| 1000/1000 [00:01<00:00, 506.99it/s]


KeyboardInterrupt: 

In [11]:
tmp = np.array(results[0])
dims = (num_simulations, n)
names = ["Influential", "Normal"]

S_influential = np.reshape(tmp[:,0], dims)
t_influential = np.reshape(tmp[:,1], dims)
S_normal = np.reshape(tmp[:,2], dims)
t_normal = np.reshape(tmp[:,3], dims)

# Averaged Time of Influenced Nodes
T_influential = np.apply_along_axis(np.mean, 0, t_influential)
T_normal = np.apply_along_axis(np.mean, 0, t_normal)

# Number of Nodes of Network Influenced
N_influential = np.apply_along_axis(np.mean, 0, S_influential)
N_normal = np.apply_along_axis(np.mean, 0, S_normal)

# Proportion of Network Influenced
P_influential = [x/N for x in N_influential]
P_normal = [x/N for x in N_normal]

NameError: name 'results' is not defined

### Plotting

In [ ]:
plt.plot(n_avg, N_influential, "-s")
plt.plot(n_avg, N_normal, "-o")
plt.ylabel("Average Number Influenced")
plt.xlabel("Average Degree")
plt.title("Number of Nodes Influenced")
plt.legend(names)

In [ ]:
plt.plot(n_avg, P_influential, "-o")
plt.plot(n_avg, P_normal, "-o")
plt.ylabel("Average Number Influenced")
plt.xlabel("Average Degree")
plt.title("Percentage of Network Influenced")
plt.legend(names)

In [ ]:
plt.plot(n_avg, T_influential, "-o")
plt.plot(n_avg, T_normal, "-o")
plt.ylabel("Average Time of Influenced")
plt.xlabel("Average Degree")
plt.title("Comparison of Average Influenced Time against Average Degree")
plt.legend(names)